In [ ]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
results_dir = f"../artifacts/results/"

list_of_datasets = ['fashion', 'cifar10', 'gtsrb',  'medmnist', 'speech', 'activity'] #'robofi', 

black_box_list = ["True", "False"]
black_box_name_list = {"True" : "Black-box", "False" : "White-box"}


list_of_detectors = ['MagNet_JSD', 'MagNet_L1', 'Manda', 'Artifacts', 'KNN', 'KNN_L1', 'GMM', 'STD', 'Max']
list_of_detectors = ['MagNet_JSD', 'Artifacts',  'Manda',  'GMM']
list_of_adv_attacks = ['FGSM', 'PGD', 'BIM',  'UAP', 'Square', ] 
list_of_bck_attacks = ['BadNet']

attack_type_list = ["adversarial"] #, "backdoor"]
eval_type_list = [0] 
eval_type_name_list = ["All"] #, "Strong", "Weak"]


In [ ]:
def get_result_dir(dataset):
    if dataset == 'mnist':
        result_dir = f"{results_dir}/{dataset}/Feat_False_128_CAE_cnn_256_3_0.5_False"
    if dataset == 'fashion':
        result_dir = f"{results_dir}/{dataset}/Feat_False_128_CAE_cnn_256_3_0.5_False"
    if dataset == 'cifar10':
        result_dir = f"{results_dir}/{dataset}/Feat_False_256_CAE_cnn_1024_4_0.1_False"
    if dataset == 'gtsrb':
        result_dir = f"{results_dir}/{dataset}/Feat_False_256_CAE_cnn_1024_5_0.1_False"
    if dataset == 'speech':
        result_dir = f"{results_dir}/{dataset}/Feat_False_1024_CAE_cnn_256_3_0.1_False"
    if dataset == 'robofi':
        result_dir = f"{results_dir}/{dataset}/Feat_False_512_CAE_cnn_1024_4_0.05_False"
    if dataset == 'medmnist':
        result_dir = f"{results_dir}/{dataset}/Feat_False_128_CAE_cnn_1024_3_0.1_False"
    if dataset == 'activity':
        result_dir = f"{results_dir}/{dataset}/Feat_False_512_CAE_cnn_1024_4_0.05_False"
    return result_dir



In [ ]:
df_total = pd.DataFrame([])

for dataset in list_of_datasets:
    for black_box in black_box_list:
        for attack_type in attack_type_list:
             for eval_type in eval_type_list:
                # print(dataset, black_box)
                result_dir = get_result_dir(dataset)
                detection_dir = f"{result_dir}/detection/final_scores_{attack_type}_{black_box}_{eval_type}_BenPerturb_100_1.csv"
                try:
                    df = pd.read_csv(detection_dir, index_col=0)
                    df['BlackBox'] = black_box_name_list[black_box]
                    df['Dataset'] = dataset
                    df["attack_type"] = attack_type
                    df["eval_type"] = eval_type_name_list[eval_type]
                    df_total = pd.concat([df_total, df], axis=0)
                    # print("File loaded at: ", detection_dir)

                except Exception as e:
                    print(e)
                # print(df_total.shape)

In [ ]:
# col_of_interest = ['Detector', 
#        'Attack', 'AUROC', 'AUPRC', 'Precision', 'Recall', 'FPR',
#        'F1Score', 'BlackBox', 'Dataset', 'attack_type', 'eval_type']
col_of_interest = ['Detector', 
       'Attack', 'AUROC', 'BlackBox', 'Dataset', 'attack_type', 'eval_type']
df_total = df_total[col_of_interest]
df_total.head(-10)

In [ ]:
filter_detector = df_total['Detector'].isin(list_of_detectors)
filter_attack = df_total['Attack'].isin(list_of_adv_attacks)
df_clear = df_total.where(filter_attack & filter_detector).dropna()
df_clear.head(10)

In [ ]:
attack_type = 'adversarial' # Adversarial attack
eval_type = "All" #"Strong" # Successful
BlackBox = "Whitebox"
dataset = 'mnist'
df_clear

In [ ]:

filter_1 = df_clear["attack_type"] == attack_type
filter_2 = df_clear["eval_type"] == "All"
# filter_2 = df_clear["eval_type"] != "Weak"
# filter_3 = df_clear["BlackBox"] == BlackBox
df_cut = df_clear.where(filter_1 & filter_2).dropna()
#df_cut = df_clear.where(filter_3).dropna()
df_cut

In [ ]:
merged_result = pd.DataFrame([])
for dataset in list_of_datasets:
    for detector in list_of_detectors:
        for blackbox in list(black_box_name_list.values())[::-1]:
            # for eval_type in ['Strong', 'Weak']:
            for eval_type in ['All']:
                for attack in list_of_adv_attacks:
                    # print(f"{dataset[0]}_{detector}_{attack}_{eval_type}_{blackbox[0]}")                
                    try:
                        auroc = df_cut[df_cut["Detector"] == detector]
                        auroc = auroc[auroc["BlackBox"] == blackbox]
                        auroc = auroc[auroc["Attack"] == attack]
                        auroc = auroc[auroc["Dataset"] == dataset]
                        auroc = auroc[auroc["eval_type"] == eval_type]
                        auroc = round(auroc['AUROC'].values[0],2)
                        auroc = "{:0.2f}".format(auroc)
                    except IndexError:
                        auroc = "-0.01"
                    attack = attack.replace("&", '')
                    merged_result.loc[f"{dataset[0:5]}-{detector}", f"{attack}-{eval_type[0]}-{blackbox[0]}"] = auroc
merged_result


In [ ]:
filtered_indices = merged_result.index.str.contains(r'GMM', regex=True)
filtered_indices

In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 7.5))
sns.heatmap(merged_result.astype(float), annot=True, fmt='0.2f', cmap="RdYlGn")
plt.show()

In [ ]:
print(df.to_latex())
table_text = df.to_latex()

In [ ]:
replacements = {
    "fashion": "F-MNIST", 
    "mnist": "MNIST", 
    "cifar10": "CIFAR-10", 
    "gtsrb": "GTSRB", 
    "speech": "Speech", 
    "medmnist" : "Med-MNIST", 
    "MagNet_JSD": "MagNet",
    "activity": "Activity",
    "GMM": "\sysname",
    }


In [ ]:
for dataset_ext in list_of_datasets:
    print(rf"\hline\hline")
    print(rf"\multirow{{5}}{{*}}{{\rotatebox{{90}}{{{replacements[dataset_ext]}}}}}")
            # print(dataset_ext)
    for lines in table_text.split("\n"):

        if dataset_ext[0:5] in lines:
            lines=lines.replace(dataset_ext[0:5]+"-", " & ")
            for key, val in replacements.items():
                lines = lines.replace(key, val)
            print(lines)


# Create Attack Data

In [ ]:
dataset_dict = {"mnist" : "MNIST",
                "fashion" : "F-MNIST",
                "cifar10" : "CIFAR-10",
                "gtsrb" : "GTSRB",
                "activity" : "Activity",
                "speech" : "Speech",
                "medmnist" : "Med-MNIST",}

In [ ]:

df_total = pd.DataFrame([])

for dataset in list_of_datasets:
    for black_box in ['White-box', 'Black-box']:
        print(dataset, black_box)
        result_dir = get_result_dir(dataset)
        detection_dir = f"{result_dir}/data/attack_gen_acc_norm_{black_box}_clean.csv"
        try:
            df = pd.read_csv(detection_dir, index_col=0)
            df['ASR'] = 100 - df['Acc']
            df = df.reset_index(drop=True)
            df['Dataset'] = dataset
            df['Access'] = black_box
            df_total = pd.concat([df_total, df], axis=0)
            # print("File loaded at: ", detection_dir)
        except Exception as e:
            print(e)
        
df_total = df_total.rename(columns={"L2" : "L2 Norm", "Acc" : "Accuracy"})
df_total

In [ ]:
df_total = df_total.replace("medmnist", "Med-MNIST")
df_total.replace(dataset_dict, inplace= True)
print(df_total[['ASR', 'Attack', 'Dataset', 'Access']])

In [ ]:
filter_1 = df_total['Attack'] != 'RN'
filter_2 = df_total['Attack'] != 'C&W'
df_total = df_total.where(filter_1 & filter_2).dropna()
df_total

In [ ]:
plt.figure(figsize=(20,3))
# g = sns.catplot(data=df_total, col="Dataset", x="Attack", y="ASR", 
#                 hue="Access", hue_order=['White-box', 'Black-box'], 
#                 kind='bar', aspect=1.25, height=2, ci=None)

g = sns.catplot(data=df_total, col="Dataset", x="Attack", y="ASR", 
            hue="Access", hue_order=['White-box', 'Black-box'], 
            kind='bar', aspect=1.05, height=2.50, errorbar=None, 
            # palette={'White-box': 'gray', 'Black-box': 'black'})  # Define colors for each hue
            # palette={'White-box': 'darkblue', 'Black-box': 'lightblue'})
            palette={'White-box': 'lightcoral', 'Black-box': 'darkred'})  # Dark and light red shades

# Add a dark boundary to the bars
for ax in g.axes.flat:  # Iterate over each subplot
    for patch in ax.patches:  # Iterate over each bar
        patch.set_edgecolor('black')  # Set the boundary color
        patch.set_linewidth(0.50)      # Set the boundary thickness

# Add horizontal lines
for ax in g.axes.flat:  # Iterate over each subplot
    ax.yaxis.grid(True)  # Enable horizontal grid lines
    # ax.axhline(0, color='black', linewidth=0.25)  # Optionally add a line at y=0 (x-axis)


g.set_axis_labels("ASR", "Attack")  # Customize axis labels if needed
# g.despine(left=True)  # Remove the left spine
g.set_titles("{col_name}")  # Optional: Title formatting for columns
# g.map(plt.grid, True)  # Enable gridlines


# plt.tight_layout()
plot_dir = Path(f"../artifacts/plots/asr_adversarial_attacks.pdf")
plot_dir.parent.mkdir(exist_ok=True, parents=True)
plt.savefig(plot_dir, dpi = 350)
plt.savefig(plot_dir)
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
# g = sns.catplot(data=df_total, col="Dataset", x="Attack", y="ASR", 
#                 hue="Access", hue_order=['White-box', 'Black-box'], 
#                 kind='bar', aspect=1.25, height=2, ci=None)

g = sns.catplot(data=df_total, col="Dataset", y="Attack", x="L2 Norm", 
            hue="Access", hue_order=['White-box', 'Black-box'], 
            kind='bar', aspect=0.75, height=3, errorbar=None, 
            # palette={'White-box': 'gray', 'Black-box': 'black'})  # Define colors for each hue
            palette={'White-box': 'darkblue', 'Black-box': 'lightblue'})
            # palette={'White-box': 'lightcoral', 'Black-box': 'darkred'})  # Dark and light red shades

# Add a dark boundary to the bars
for ax in g.axes.flat:  # Iterate over each subplot
    for patch in ax.patches:  # Iterate over each bar
        patch.set_edgecolor('black')  # Set the boundary color
        patch.set_linewidth(0.50)      # Set the boundary thickness

# Add horizontal lines
for ax in g.axes.flat:  # Iterate over each subplot
    ax.yaxis.grid(True)  # Enable horizontal grid lines
    # ax.axhline(0, color='black', linewidth=0.25)  # Optionally add a line at y=0 (x-axis)


g.set_axis_labels("L2 Norm", "Attack")  # Customize axis labels if needed
# g.despine(left=True)  # Remove the left spine
g.set_titles("{col_name}")  # Optional: Title formatting for columns
# g.map(plt.grid, True)  # Enable gridlines


# plt.tight_layout()
# plt.savefig(f"../artifacts/plots/norm_adversarial_attacks.jpg", dpi = 350)
# plt.savefig(f"../artifacts/plots/norm_adversarial_attacks.pdf")
plot_dir = Path(f"../artifacts/plots/norm_adversarial_attacks.pdf")
plot_dir.parent.mkdir(exist_ok=True, parents=True)
plt.savefig(plot_dir, dpi = 350)
plt.savefig(plot_dir)
plt.show()